# Deep Neural Network Model Design

**By: M. Alwi Sukra**

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from matplotlib import pyplot as plt

### 1. Get Data

In [2]:
DATA_PATH = "datasets/ann-all"

In [3]:
X_train = np.load(f"{DATA_PATH}-X_train.npy")
y_train = np.load(f"{DATA_PATH}-y_train.npy")
X_test = np.load(f"{DATA_PATH}-X_test.npy")
y_test = np.load(f"{DATA_PATH}-y_test.npy")

In [4]:
X_train[23]

array([0.70747894, 0.62666669, 0.08818361, 0.04614529, 0.        ,
       0.        ])

In [5]:
y_train[23]

0

### 2. Build Model

In [6]:
MODEL_NAME = "DNN_all-features"

In [7]:
layers = [3,3,3]

#### Model Structure

In [8]:
model = tf.keras.models.Sequential()
for neurons in layers:
    model.add(tf.keras.layers.Dense(neurons, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

#### Compile Model

In [9]:
opt = tf.optimizers.Adam(lr=0.01)
acc = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=[acc])

### 3. Train Model

In [10]:
trained_on = int(time.time())

In [11]:
EPOCHS = 50
VALIDATION_SPLIT = 0.2

#### Callbacks Initialization

In [12]:
checkpoint_filepath = f"saved-models/{trained_on}_{MODEL_NAME}" + "_ep-{epoch:02d}_valos-{val_loss:.4f}.hdf5"
log_filepath = f"csv-train-logs/{trained_on}_{MODEL_NAME}.csv"

In [13]:
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=0, mode='min', save_best_only=True)

In [14]:
csv_log = CSVLogger(log_filepath, separator=',', append=False)

In [15]:
callbacks=[checkpoint, csv_log]

#### Training Process

In [16]:
model.fit(X_train, y_train, validation_split=VALIDATION_SPLIT, epochs=EPOCHS, verbose=2, callbacks=callbacks)

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1131/1131 - 3s - loss: 0.3961 - binary_accuracy: 0.8105 - val_loss: 0.3556 - val_binary_accuracy: 0.8372
Epoch 2/50
1131/1131 - 3s - loss: 0.3549 - binary_accuracy: 0.8304 - val_loss: 0.3534 - val_binary_accuracy: 0.8099
Epoch 3/50
1131/1131 - 3s - loss: 0.3495 - binary_accuracy: 0.8294 - val_loss: 0.3516 - val_binary_accuracy: 0.7990
Epoch 4/50
1131/1131 - 3s - loss: 0.3454 - binary_accuracy: 0.8319 - val_loss: 0.3397 - val_binary_accuracy: 0.8515
Epoch 5/50
1131/1131 - 3s - loss: 0.3402 - b